# Build Tiled Library

To speed up mapping, we will tile segment-based library where the FSP-FPP relations are organized by tiles. This task includes the following main steps:

* Tile segment-based library
* Calculate FSP and segment downstream distance
* Assign stream order to segments and FSPs

## Import Python Packages

In [7]:
import os

# import the tile module from the fldpln package
from fldpln.tile import *

## Tile Library

Here we spatially divide a segment-based library into tiles. Note that tile size is the number of cells to avoid partial cells within a tile and can be used for both PCS and GCS. 

This process also copies the FSP and segment info CSV files to the tiled library and creates a metadata file (TileCellSizeSpatialReference.json) which stores library tile and cell sizes and spatial reference in a JSON file.

**Note that this is the most time consuming process and may take hours for large libraries.**

In [ ]:
# Segment-based library
segLibFolder = 'E:/fldpln/sites/wildcat_10m_3dep/seglib'

# tiled library folder
tiledLibFolder = 'E:/fldpln/sites/wildcat_10m_3dep/tiledlib' 

# define tile size (number of cells) and format
cellSize = 10
tileSize = 500 # number of cells
tileFileFormat = 'snappy' # 'snappy' or 'mat'

# tile the library
print(f'Tile library: {segLibFolder} ...')
TileLibrary(segLibFolder, cellSize, tiledLibFolder,tileSize,tileFileFormat) 

## Calculate FSP and Segment Downstream Distance

Here we calculate downstream distance for both FSPs and segments for use in inundation mapping. It involves the following major sub-tasks:
* Clean up segments 
    * removes segments from the segment table if they are not in the FSP table. 
    * If a removed segment is the downstream segment of another segment in the segment table, the upstream segment ID is set to 0 (i.e., watershed outlet). 
    * Those removed segments are usually close to or in waterbodies. By removing those segments, a library may have several separate watersheds/outlets! For example, neosho has 3 separate watersheds (segment 13, 104, 186 as the outlet segments). 
* Calculate FSP and segment downstream distance (i.e., distance from outlet) using in interpolating FSP depth of water from gauges

At the end, this step updates the fsp_info and segment_info CSV files by adding additional columns. 

In [ ]:
# tiled library folder
tiledLibFolder = 'E:/fldpln/sites/wildcat_10m_3dep/tiledlib' 

print(f'Calculate FSP and segment downstream distance for tiled library: {tiledLibFolder} ...')
CalculateFspSegmentDownstreamDistance(tiledLibFolder)

## Assign Stream Orders to FSP and Segment CSV Files

This step gets stream orders from a segment shapefile, where each segment is assigned an order automatically or manually, and add the stream orders to their FSPs and segments in the fsp_info and segment_info CSV files. 

It also creates a new text file, stream_order_info.csv, which stores the connectivity among stream orders with columns: [‘StrOrd’, ‘DsStrOrd’, ‘JunctionFspX’, ‘JunctionFspY’]. This information is used in DOF interpolation.

In [ ]:
# tiled library folder to add stream order
tiledLibFolder= 'E:/fldpln/sites/wildcat_10m_3dep/tiledlib'

# FSP and segment info CSV files
segInfoFile = os.path.join(tiledLibFolder,segInfoFileName)
fspInfoFile = os.path.join(tiledLibFolder,fspInfoFileName)

# # segment shapefile which has the stream order manually assigned
# shpFile = 'E:/fldpln/sites/wildcat_10m_3dep/segment_shapefiles/wildcat_segments.shp'
# shpSegIdColName = 'grid_code'
# shpOrdColName = 'str_ord'
# segment shapefile which has the stream order automatically assigned
shpFile = 'E:/fldpln/sites/wildcat_10m_3dep/segs/wildcat_segments_ord4map.shp'
shpSegIdColName = 'segid' # or 'grid_code'
shpOrdColName = 'strord' # strord = 'subnetwork' * 1000 + 'snetstrord'

print(f'Get stream order and generate stream order network info for : {tiledLibFolder} ...')
GetStreamOrdersForFspsSegments(tiledLibFolder,shpFile,shpSegIdColName,shpOrdColName)

## Additional Utilities

### Generate Segment Shapefile

Create a segment shapefile from segment_info.csv file. Note that the shapefile gets its CRS from the library metadata file and has all the attributes in the segment_info CSV file.

In [ ]:
# import CRS from pyproj for reading WKT
from pyproj import CRS

# tiled library folder to generate segment shapefile
tiledLibFolder= 'E:/fldpln/sites/wildcat_10m_3dep/tiledlib'

# FSP and segment info CSV files
segInfoFile = os.path.join(tiledLibFolder,segInfoFileName)
fspInfoFile = os.path.join(tiledLibFolder,fspInfoFileName)

# Read lib CRS from metadata file
metaDataFile = os.path.join(tiledLibFolder,metaDataFileName)
with open(metaDataFile,'r') as jf:
    md = json.load(jf)
srText = md['SpatialReference']
libCrs = CRS.from_wkt(srText)

# segment shapefile folder
shpFolder = 'E:/fldpln/sites/wildcat_10m_3dep/segment_shapefiles'
shpName = "py_segments.shp"
outShpFile =  os.path.join(shpFolder,shpName)

# generate segment shapefile 
print(f'Generate segment shapefile for libraries ...')
# GenerateSegmentShapefiles(tiledLibFolder,libNames,shpFolder,shpName)
GenerateSegmentShapefilesFromFspSegmentInfoFiles(segInfoFile, fspInfoFile, libCrs, outShpFile)